In [ ]:
## IMPORTACION DE BIBLIOTECAS
import pandas as pd
from sklearn.neighbors import BallTree
import numpy as np
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import altair as alt
import webbrowser
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

In [ ]:
## Leo el dataset
url = 'https://raw.githubusercontent.com/NazarethEliasNalbandian/Analisis_Precio_Surtidor/main/data/precios_nafta.csv'
df = pd.read_csv(url)

In [ ]:
## ANÁLISIS EXPLORATORIO

In [ ]:
## FILTRADO DE DATOS

In [ ]:
## ELIMINO IDEMPRESA, CUIT, IDPRODUCTO, IDTIPOHORARIO, IDEMPRESABANDERA
print(f"Cantidad de registros en el dataset original: {df.shape[0]}")

columns_to_drop = ['idempresa', 'cuit', 'idproducto', 'idtipohorario', 'idempresabandera']
df_cleaned = df.drop(columns=columns_to_drop)

print(f"Cantidad de registros en el dataset modificado: {df_cleaned.shape[0]}")

df_cleaned.to_csv('/Users/nazarethnalbandian/Desktop/TP NAFTA/precios_nafta.csv', index=False)

print("El archivo 'precios_nafta.csv' ha sido sobrescrito con el dataset modificado.")

In [ ]:
## ELIMINO TIPO HORARIO NO DIURNO Y COLUMNA TIPOHORARIO
df_diurno = df[df['tipohorario'] == 'Diurno']
df_diurno.drop(columns=['tipohorario'], inplace=True)

print(f"Cantidad de registros en el dataset original: {df.shape[0]}")
print(f"Cantidad de registros en el dataset modificado (solo diurno): {df_diurno.shape[0]}")

df_diurno.to_csv(file_path, index=False)

print("El archivo 'precios_nafta.csv' ha sido sobrescrito con el dataset modificado.")


In [ ]:
## ELIMINO PRECIOS OUTLIERS TENIENDO EN CUENTA EL TIPO DE PRODUCTO
df['latitud'] = pd.to_numeric(df['latitud'], errors='coerce')
df['longitud'] = pd.to_numeric(df['longitud'], errors='coerce')

df = df.dropna(subset=['latitud', 'longitud'])

def eliminar_outliers_por_producto(df):
    mean_price = df['precio'].mean()
    std_price = df['precio'].std()
    
    threshold_upper = mean_price + 3 * std_price
    threshold_lower = mean_price - 3 * std_price
    
    df_cleaned = df[(df['precio'] <= threshold_upper) & (df['precio'] >= threshold_lower)]
    return df_cleaned

dataset_cleaned = df.groupby('producto').apply(eliminar_outliers_por_producto).reset_index(drop=True)

print(f"Dataset original: {df.shape[0]} registros")
print(f"Dataset limpio: {dataset_cleaned.shape[0]} registros (outliers eliminados)")

dataset_cleaned.to_csv('precios_nafta.csv', index=False)

print("El archivo 'precios_nafta.csv' ha sido sobrescrito con el dataset limpio.")

In [ ]:
## ELIMINO REGISTROS QUE NO SEAN ACTUALES (ANTERIORES A AGOSTO DE 2024)
df_august_september_2024 = df[df['indice_tiempo'].isin(['2024-08', '2024-09'])]

print(f"Cantidad de registros en el dataset original: {df.shape[0]}")

print(f"Cantidad de registros en el dataset de agosto y septiembre de 2024: {df_august_september_2024.shape[0]}")

df_august_september_2024.to_csv('precios_nafta.csv', index=False)

print("El archivo 'precios_nafta.csv' ha sido sobrescrito con los registros filtrados de agosto y septiembre de 2024.")

In [ ]:
## MINADO DE DATOS

In [ ]:
## AGREGO COLUMNA ABREVIATURA PROVINCIA EN BASE A LA COLUMNA PROVINCIA
province_abbreviations = {
    'BUENOS AIRES': 'BA',
    'CAPITAL FEDERAL': 'CABA',
    'CATAMARCA': 'C',
    'CHACO': 'CHO',
    'CHUBUT': 'Cht',
    'CORDOBA': 'CBA',
    'CORRIENTES': 'Cts',
    'ENTRE RIOS': 'ER',
    'FORMOSA': 'F',
    'JUJUY': 'J',
    'LA PAMPA': 'LP',
    'LA RIOJA': 'LR',
    'MENDOZA': 'Mza',
    'MISIONES': 'Ms',
    'NEUQUEN': 'N',
    'RIO NEGRO': 'RN',
    'SALTA': 'S',
    'SAN JUAN': 'SJ',
    'SAN LUIS': 'SL',
    'SANTA CRUZ': 'SC',
    'SANTA FE': 'SF',
    'SANTIAGO DEL ESTERO': 'SE',
    'TIERRA DEL FUEGO': 'TAIS',
    'TUCUMAN': 'T'
}

df['abreviatura_provincia'] = df['provincia'].map(province_abbreviations)

df.to_csv('precios_nafta.csv', index=False)

In [ ]:
## AGREGO CANTIDAD DE ESTACIONES ALREDEDOR DE UN RADIO DE 1 KM EN BASE A LAS COLUMNAS DE LATITUD Y LONGITUD
df['latitud_rad'] = np.deg2rad(df['latitud'])
df['longitud_rad'] = np.deg2rad(df['longitud'])

coords = np.array(df[['latitud_rad', 'longitud_rad']])

tree = BallTree(coords, metric='haversine')

radio_km = 1
radio_radianes = radio_km / 6371.0

df['cantidad_estaciones_alrededor'] = tree.query_radius(coords, r=radio_radianes, count_only=True) - 1

df.to_csv('precios_nafta.csv', index=False)

In [ ]:
## VISUALIZACION

In [ ]:
## VARIACION DE PRECIO SEGUN LA PROVINCIA Y EL TIPO DE PRODUCTO

In [ ]:
## MAPA INTERACTIVO CON CADA ESTACION EN CADA PROVINCIA CUYO COLOR QUE REPRESENTA SU PRECIO
df['latitud'] = pd.to_numeric(df['latitud'], errors='coerce')
df['longitud'] = pd.to_numeric(df['longitud'], errors='coerce')

df = df.dropna(subset=['latitud', 'longitud'])

df = df.query("producto != 'GNC'")

norm = plt.Normalize(df['precio'].min(), df['precio'].max())

mapa = folium.Map(location=[-38.4161, -63.6167], zoom_start=4, tiles='CartoDB positron')

for idx, row in dataset.iterrows():
    color = plt.cm.RdYlGn_r(norm(row['precio']))

    color_hex = mcolors.to_hex(color)
    
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=5,
        color=None,
        fill=True,
        fill_color=color_hex,
        fill_opacity=0.85 if color_hex != '#ffffbf' else 0,
        popup=(
            f"<b>Precio:</b> ${row['precio']}<br>"
            f"<b>Producto:</b> {row['producto']}"
        )
    ).add_to(mapa)

mapa.save('VISUALIZACION/mapa_interactivo_estaciones_colores.html')

print("Mapa interactivo creado y guardado como 'mapa_interactivo_estaciones_colores.html'")

In [ ]:
## GRAFICO DE BARRAS DEL PRECIO PROMEDIO POR PRODUCTO EN CADA PROVINCIA
avg_price_per_province = df.groupby(['provincia', 'producto']).agg({'precio': 'mean'}).reset_index()
avg_price_per_province['precio'] = avg_price_per_province['precio'].round(2)

province_abbreviations = {
    'BUENOS AIRES': 'BA',
    'CAPITAL FEDERAL': 'CABA',
    'CATAMARCA': 'C',
    'CHACO': 'CHO',
    'CHUBUT': 'Cht',
    'CORDOBA': 'CBA',
    'CORRIENTES': 'Cts',
    'ENTRE RIOS': 'ER',
    'FORMOSA': 'F',
    'JUJUY': 'J',
    'LA PAMPA': 'LP',
    'LA RIOJA': 'LR',
    'MENDOZA': 'Mza',
    'MISIONES': 'Ms',
    'NEUQUEN': 'N',
    'RIO NEGRO': 'RN',
    'SALTA': 'S',
    'SAN JUAN': 'SJ',
    'SAN LUIS': 'SL',
    'SANTA CRUZ': 'SC',
    'SANTA FE': 'SF',
    'SANTIAGO DEL ESTERO': 'SE',
    'TIERRA DEL FUEGO': 'TAIS',
    'TUCUMAN': 'T'
}

avg_price_per_province['provincia_abbr'] = avg_price_per_province['provincia'].map(province_abbreviations)

chart = alt.Chart(avg_price_per_province).mark_bar().encode(
    x=alt.X('provincia_abbr:N', title='Provincias', sort=list(province_abbreviations.values())),
    y=alt.Y('precio:Q', title='Precio Promedio'),
    color=alt.Color('producto:N', title='Producto'),
    tooltip=[alt.Tooltip('provincia:N', title='Provincia'), alt.Tooltip('precio:Q', title='Precio Promedio')]
).properties(
    title='Precio Promedio de Cada Tipo de Combustible en Cada Provincia',
    width=alt.Step(20)
).facet(
    column=alt.Column('producto:N', title='Producto')
).configure_axis(
    labelAngle=0
).configure_facet(
    spacing=10
).configure_view(
    stroke=None
)

chart.save('VISUALIZACION/grafico_precio_promedio_por_producto.html')
webbrowser.open('VISUALIZACION/grafico_precio_promedio_por_producto.html')


In [ ]:
## VARIACION DE LOS PRECIOS DE AGOSTO A SEPTIEMBRE

In [ ]:
## GRAFICO DE BARRAS QUE MUESTRA EL PRECIO PROMEDIO DE AGOSTO Y SEPTIEMBRE POR PRODUCTO PARA CADA PROVINCIA
df_august_september = df[df['indice_tiempo'].isin(['2024-08', '2024-09'])]

avg_price_per_product_province_month = df_august_september.groupby(['producto', 'abreviatura_provincia', 'indice_tiempo']).agg({'precio': 'mean'}).reset_index()
avg_price_per_product_province_month['precio'] = avg_price_per_product_province_month['precio'].round(2)

chart = alt.Chart(avg_price_per_product_province_month).mark_bar().encode(
    x=alt.X('abreviatura_provincia:N', title='Provincias', sort=alt.EncodingSortField(field='abreviatura_provincia', op='count')),
    y=alt.Y('precio:Q', title='Precio Promedio'),
    color=alt.Color('indice_tiempo:N', title='Mes', scale=alt.Scale(domain=['2024-08', '2024-09'], range=['#1f77b4', '#ff7f0e'])),
    column=alt.Column('producto:N', title='Producto', spacing=100),
    xOffset='indice_tiempo:N',
    tooltip=[alt.Tooltip('abreviatura_provincia:N', title='Provincia'), alt.Tooltip('precio:Q', title='Precio Promedio'), alt.Tooltip('indice_tiempo:N', title='Mes')]
).properties(
    title='Comparación del Precio Promedio de Combustible en Agosto y Septiembre por Provincia y Producto',
    width=300,
    height=600
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=20
).resolve_scale(
    y='independent'
)

chart.save('VISUALIZACION/grafico_precio_promedio_agosto_septiembre_por_producto.html')

webbrowser.open('VISUALIZACION/grafico_precio_promedio_agosto_septiembre_por_producto.html')

In [ ]:
## VARIACION DE PRECIO EN UNA ESTACION EN BASE A SU CERCANÍA CON OTRAS ESTACIONES

In [ ]:
## GRAFICO MERIMEKKO DE LAS PROVINCIAS EN CADA PRODUCTO Y
## SCATTERPLOT QUE BUSCA LA CORRELACION ENTRE EL PRECIO DE LA ESTACION Y LA CANTIDAD DE ESTACIONES ALREDEDOR DE 1KM 
app = Dash(__name__)

avg_price_per_product_province = df.groupby(['producto', 'provincia']).agg({'precio': 'mean'}).reset_index()
avg_price_per_product_province['precio'] = avg_price_per_product_province['precio'].round(2)

fig = px.treemap(
    avg_price_per_product_province,
    path=['producto', 'provincia'],
    values='precio',
    color='precio',
    color_continuous_scale='Viridis',
    title='Distribución de Precios Promedios de Combustible por Producto y Provincia',
    hover_data={'precio': ':.2f'},
    custom_data=['precio'],
    height=800,
    width=1200
)

fig.update_traces(
    hovertemplate='Precio: %{customdata[0]:.2f}<extra></extra>',
    texttemplate='%{label}<br>Precio: %{customdata[0]:.2f}', 
    textposition='middle center' 
)

app.layout = html.Div([
    dcc.Graph(id='treemap', figure=fig),
    dcc.Graph(id='scatter-plot')
])

@app.callback(
    Output('scatter-plot', 'figure'),
    Input('treemap', 'clickData')
)
def update_scatter_plot(clickData):
    if clickData:
        producto = clickData['points'][0]['id'].split('/')[0]
        provincia = clickData['points'][0]['id'].split('/')[1]
        
        df_filtered = df[(df['producto'] == producto) & (df['provincia'] == provincia)]
        
        scatter_plot = px.scatter(
            df_filtered,
            x='cantidad_estaciones_alrededor',
            y='precio',
            color='cantidad_estaciones_alrededor',
            color_continuous_scale='Viridis',
            title=f'Correlación entre Cantidad de Estaciones Alrededor y Precio en {provincia} para {producto}',
            labels={'cantidad_estaciones_alrededor': 'Cantidad de Estaciones Alrededor', 'precio': 'Precio'},
            hover_data={'precio': True, 'cantidad_estaciones_alrededor': True}
        )
        scatter_plot.update_traces(
            hovertemplate='Cantidad de Estaciones Alrededor: %{x}<br>Precio: %{y}<extra></extra>'
        )
        return scatter_plot
    else:
        return {}

if __name__ == '__main__':
    app.run_server(debug=True)